## IBM Data Science Capstone Assignment- Week3- Part2

### Importing the libraries


In [2]:

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### Scraping Data from Wikipedia page and converting in to a Pandas DataFrame

In [64]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [65]:
soup = BeautifulSoup(source, 'lxml') #Parsing the source data from the html file in to soup object

In [67]:
#Creating empty list for Postal code, Borrough, Neighborhood to add the Web Data to and then add to the Pandas DataFrame
PostalCode = []
Borrough = []
Neighbourhood = []

In [68]:
for row in soup.find('table').find_all('tr'): #find_all will return a list of table rows from first table on webpage
    td = row.find_all('td') #this returns list of table data
    print(td)
        
    
#Printing all the table data from table body  

[]
[<td>M1A</td>, <td>Not assigned</td>, <td>Not assigned
</td>]
[<td>M2A</td>, <td>Not assigned</td>, <td>Not assigned
</td>]
[<td>M3A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td>]
[<td>M4A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td>]
[<td>M5A</td>, <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>, <td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
</td>]
[<td>M6A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
</td>]
[<td>M6A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Lawrence_Manor" title="Lawrence Manor">Lawrence Manor</a>
</td>]
[<td>M7A</td>, <

In [69]:
for row in soup.find('table').find_all('tr'):
    td = row.find_all('td')
    if len(td)>0: #since the first list is empty using this if statement
        PostalCode.append(td[0].text) #Appending only the text part of each table data
        Borrough.append(td[1].text)
        Neighbourhood.append(td[2].text.rstrip('\n'))

In [70]:
# create a new DataFrame from the three lists
toronto_df = pd.DataFrame({"PostalCode": PostalCode,
                           "Borough": Borrough,
                           "Neighborhood": Neighbourhood})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Removing those rows from DataFrame that have Not assigned values for Borough column

In [71]:
toronto_df = toronto_df[toronto_df['Borough']!= 'Not assigned']
toronto_df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


### Grouping the DataFrame as per the PostalCode and Borough column. Combining Neighborhood with a ',' separator

In [72]:
toronto_df_group = toronto_df.groupby(['PostalCode', 'Borough'], as_index = False).agg({'Neighborhood': lambda x: ' , '.join(x)})

In [87]:
toronto_df_group.head(15)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge , Malvern"
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park , Ionview , Kennedy Park"
7,M1L,Scarborough,"Clairlea , Golden Mile , Oakridge"
8,M1M,Scarborough,"Cliffcrest , Cliffside , Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff , Cliffside West"


In [74]:
toronto_df_group[toronto_df_group['Neighborhood']=='Not assigned']

,PostalCode,Borough,Neighborhood
93,M9A,Queen's Park,Not assigned


### Changing the name of Neighborhood with value of 'Not assigned' to the name of the Borough.

In [78]:
for x in toronto_df_group['Neighborhood']:
    if x == 'Not assigned':
        toronto_df_group.loc[toronto_df_group['Neighborhood']=='Not assigned', 'Neighborhood'] = toronto_df_group.loc[toronto_df_group['Neighborhood']=='Not assigned', 'Borough']

### Checking to confirm if the row with Not assigned values for Neighborhood column has been changed

In [82]:
print(toronto_df_group[toronto_df_group['Neighborhood']=='Not assigned'])
toronto_df_group.iloc[93]

Empty DataFrame
Columns: [PostalCode, Borough, Neighborhood]
Index: []


PostalCode               M9A
Borough         Queen's Park
Neighborhood    Queen's Park
Name: 93, dtype: object

### Printing the number of rows in the final dataframe for toronto_df_group

In [83]:
toronto_df_group.shape

(103, 3)

### Loading the coordinates from the csv file

In [89]:
geo_coord = pd.read_csv('Geospatial_Coordinates.csv')

In [90]:
geo_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [92]:
geo_coord.rename(columns={'Postal Code': 'PostalCode'}, inplace = True)

In [95]:
geo_coord.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merging the two DataFrames to combine the Postal code, borough, neighborhood columns from toronto_df_group dataframe with latitutde and longitude columns from geo_coord dataframe.

In [96]:
# merge two table on the column "PostalCode"
final_toronto_df = toronto_df_group.merge(geo_coord, on="PostalCode", how="left")
final_toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge , Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
